# Query Pipeline with Routing

Here we showcase our query pipeline with routing.

Routing lets us dynamically choose underlying query pipelines to use given the query and a set of choices.

We offer this as an out-of-the-box abstraction in our [Router Query Engine](https://docs.llamaindex.ai/en/stable/examples/query_engine/RouterQueryEngine.html) guide. Here we show you how to compose a similar pipeline using our Query Pipeline syntax - this allows you to not only define query engines but easily stitch it into a chain/DAG with other modules across the compute graph.

We show this in two ways:
1. **Using a Router Component**: This is a Component that is composed on top of other query pipelines, and selects them based on a condition.
2. **Using Conditional Edges**: You can make the edges in a graph "conditional", meaning that they are only picked if certain conditions are met.

## Load Data

Load in the Paul Graham essay as an example.

In [ ]:
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt' -O pg_essay.txt

--2024-02-10 00:31:34--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘pg_essay.txt’

pg_essay.txt        100%[===================>]  73.28K  --.-KB/s    in 0.01s   

2024-02-10 00:31:34 (6.78 MB/s) - ‘pg_essay.txt’ saved [75042/75042]



In [ ]:
from llama_index import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["pg_essay.txt"])
documents = reader.load_data()

## Define Modules

We define llm, vector index, summary index, and prompt templates.

In [ ]:
from llama_index.query_pipeline import (
    QueryPipeline,
    InputComponent,
)
from typing import Dict, Any, List, Optional
from llama_index.llms.openai import OpenAI
from llama_index import (
    Document,
    VectorStoreIndex,
    ServiceContext,
    SummaryIndex,
)
from llama_index.response_synthesizers import TreeSummarize
from llama_index.schema import NodeWithScore, TextNode
from llama_index.prompts import PromptTemplate
from llama_index.selectors import LLMSingleSelector

# define HyDE template
hyde_str = """\
Please write a passage to answer the question: {query_str}

Try to include as many key details as possible.

Passage: """
hyde_prompt = PromptTemplate(hyde_str)

# define llm
llm = OpenAI(model="gpt-3.5-turbo")


# define synthesizer
summarizer = TreeSummarize(
    service_context=ServiceContext.from_defaults(llm=llm)
)

# define vector retriever
vector_index = VectorStoreIndex.from_documents(documents)
vector_retriever = vector_index.as_retriever(similarity_top_k=2)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=2)

# define summary query prompts + retrievers
summary_index = SummaryIndex.from_documents(documents)
summary_qrewrite_str = """\
Here's a question:
{query_str}

You are responsible for feeding the question to an agent that given context will try to answer the question.
The context may or may not be relevant. Rewrite the question to highlight the fact that
only some pieces of context (or none) maybe be relevant.
"""
summary_qrewrite_prompt = PromptTemplate(summary_qrewrite_str)
summary_retriever = summary_index.as_retriever()
summary_query_engine = summary_index.as_query_engine()

# define selector
selector = LLMSingleSelector.from_defaults()

## Setup Query Pipeline with `RouterComponent`

In the first approach, we show you how to setup a query pipeline with a `RouterComponent`. The `RouterComponent` specifically takes in one of our `Selector` modules, and given a set of choices (with string descriptions), chooses the relevant choice and calls the relevant sub-component.

### Construct Query Pipelines

Define a query pipeline for vector index, summary index, and join it together with a router.

In [ ]:
# define summary query pipeline
from llama_index.query_pipeline import RouterComponent

vector_chain = QueryPipeline(chain=[vector_query_engine])
summary_chain = QueryPipeline(
    chain=[summary_qrewrite_prompt, llm, summary_query_engine], verbose=True
)

choices = [
    "This tool answers specific questions about the document (not summary questions across the document)",
    "This tool answers summary questions about the document (not specific questions)",
]

router_c = RouterComponent(
    selector=selector,
    choices=choices,
    components=[vector_chain, summary_chain],
    verbose=True,
)
# top-level pipeline
qp = QueryPipeline(chain=[router_c], verbose=True)

### Try out Queries

In [ ]:
# compare with sync method
response = qp.run("What did the author do during his time in YC?")
print(str(response))

> Running module c0a87442-3165-443d-9709-960e6ddafe7f with input: 
query: What did the author do during his time in YC?

Selecting component 0: The author used a tool to answer specific questions about the document, which suggests that he was engaged in analyzing and extracting specific information from the document during his time in YC..
During his time in YC, the author worked on various tasks related to running Y Combinator. This included selecting and helping founders, dealing with disputes between cofounders, figuring out when people were lying, and fighting with people who maltreated the startups. The author also worked on writing essays and internal software for YC.


In [ ]:
response = qp.run("What is a summary of this document?")
print(str(response))

> Running module c0a87442-3165-443d-9709-960e6ddafe7f with input: 
query: What is a summary of this document?

Selecting component 1: The summary questions about the document are answered by this tool..
> Running module 0e7e9d49-4c92-45a9-b3bf-0e6ab76b51f9 with input: 
query_str: What is a summary of this document?

> Running module b0ece4e3-e6cd-4229-8663-b0cd0638683c with input: 
messages: Here's a question:
What is a summary of this document?

You are responsible for feeding the question to an agent that given context will try to answer the question.
The context may or may not be relev...

> Running module f247ae78-a71c-4347-ba49-d9357ee93636 with input: 
input: assistant: What is the summary of the document?

The document discusses the development and evolution of Lisp as a programming language. It highlights how Lisp was originally created as a formal model of computation and later transformed into a programming language with the assistance of Steve Russell. The document also emph

## Setup Query Pipeline with Conditional Links

In the example below we should you how to build our query pipeline with conditional links to route between our summary query engine and router query engine depending on the user query.

In [ ]:
from llama_index.query_pipeline import (
    QueryPipeline,
    InputComponent,
    Link,
    FnComponent,
)
from llama_index.selectors import LLMSingleSelector
from typing import Dict

We first initialize our `LLMSingleSelector` component. Given a set of choices and a user query it will output a selection indicating the choice it picks.

Note that the `LLMSingleSelector` can be directly used in a query pipeline. However here we wrap it in a `FnComponent` so that we can return the output as a dictionary of both the selected index and the original user query (this will help when we define our conditional link).

In [ ]:
choices = [
    "This tool answers specific questions about the document (not summary questions across the document)",
    "This tool answers summary questions about the document (not specific questions)",
]


def select_choice(query: str) -> Dict:
    selector = LLMSingleSelector.from_defaults()
    output = selector.select(choices, query)
    return {"query": query, "index": str(output.ind)}

We now initialize our Query Pipeline with the modules: input, selector, vector/summary query engine.

In [ ]:
qp = QueryPipeline(
    modules={
        "input": InputComponent(),
        "selector": FnComponent(fn=select_choice),
        "vector_retriever": vector_retriever,
        "summary_retriever": summary_retriever,
        "summarizer": summarizer,
    },
    verbose=True,
)

We now define our links. Input --> selector is standard. What's more interesting here is our conditional link. 

We input our selector component as the source. 

We then input a function that produces two outputs, the first being the condition variable and the second being the child component.

Lastly, we define a dictionary mapping each condition variable value to the component (which is represented as a dictionary with "dest" and "dest_key"). 

In [ ]:
qp.add_link("input", "selector")
qp.add_link(
    "selector",
    "vector_retriever",
    condition_fn=lambda x: x["index"] == "0",
    input_fn=lambda x: x["query"],
)
qp.add_link(
    "selector",
    "summary_retriever",
    condition_fn=lambda x: x["index"] == "1",
    input_fn=lambda x: x["query"],
)
qp.add_link("vector_retriever", "summarizer", dest_key="nodes")
qp.add_link("summary_retriever", "summarizer", dest_key="nodes")
qp.add_link("input", "summarizer", dest_key="query_str")

### Visualize

The benefit of conditional links is that 

In [ ]:
## create graph
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(qp.clean_dag)
net.show("rag_dag.html")

rag_dag.html


### Try out Queries

In [ ]:
# compare with sync method
response = qp.run(input="What did the author do during his time in YC?")
print(str(response))

> Running module input with input: 
input: What did the author do during his time in YC?

> Running module selector with input: 
query: What did the author do during his time in YC?

> Running module vector_retriever with input: 
input: What did the author do during his time in YC?

> Running module summarizer with input: 
query_str: What did the author do during his time in YC?
nodes: [NodeWithScore(node=TextNode(id_='e7937e10-61ca-435a-a6a7-4232cdd7bed8', embedding=None, metadata={'file_path': 'pg_essay.txt', 'file_name': 'pg_essay.txt', 'file_type': 'text/plain', 'file_size': 750...

During his time in YC, the author worked on various tasks related to running the program. He selected and helped founders, dealt with disputes between cofounders, figured out when people were lying, and fought with people who maltreated the startups. Additionally, he wrote all of YC's internal software and worked on other projects such as writing essays and managing Hacker News. The author worked hard a

In [ ]:
len(response.source_nodes)

2

In [ ]:
response = qp.run(input="What is a summary of this document?")
print(str(response))

> Running module input with input: 
input: What is a summary of this document?

> Running module selector with input: 
query: What is a summary of this document?

> Running module summary_retriever with input: 
input: What is a summary of this document?

> Running module summarizer with input: 
query_str: What is a summary of this document?
nodes: [NodeWithScore(node=TextNode(id_='f6d0a4c7-3806-4169-8e87-ba7870335312', embedding=None, metadata={'file_path': 'pg_essay.txt', 'file_name': 'pg_essay.txt', 'file_type': 'text/plain', 'file_size': 750...

The document provides a personal narrative of the author's journey from studying art in Florence to working at a software company in the US. It discusses their experiences at the Accademia di Belli Arti in Florence, their time working at Interleaf, their decision to drop out of art school and pursue painting and writing books on Lisp, and their failed attempt to start an online art gallery business. The document also covers the founding and 

In [ ]:
len(response.source_nodes)

21